<a href="https://colab.research.google.com/github/svondracek0/Data-Science-Practicum/blob/main/SDA_15_02_2024_Data_Processign_and_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Processing Opakování

V tomto notebooku naleznete materiály, se kterými jsme opakovali data processing pomocí knihovny pandas.

## Načtení a představení dat

In [ ]:
# import knihovny
import pandas as pd

In [ ]:
# Načtení dat
df_housing = pd.read_csv("https://raw.githubusercontent.com/melindaleung/Ames-Iowa-Housing-Dataset/refs/heads/master/data/ames%20iowa%20housing.csv")

In [ ]:
# Sloupce nasi tabulky
df_housing.columns

Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'GarageCond', 'PavedDrive

Naším cílen není konstruovat ML model, ale pouze si ukázat základní operace které můžeme vykonávat nad pd.DataFrame objektem. K ilustraci s izobrazíme proměnnou SalePrice, na které si transformace ukážeme.

In [ ]:
import plotly.express as px
fig = px.histogram(df_housing['SalePrice'], nbins=60)
fig.show(title="Cena nemovitosti v Iowa - Ames")

## Datové transformace

Vidíme, že proměnná je pozitivně sešikmená. V takovém případě se někdy hodí provést tzv. logaritmickou transformaci, kterou data log-**normalizujeme**

In [ ]:
import numpy as np

In [ ]:
import plotly.express as px
fig = px.histogram(np.log(df_housing['SalePrice']), nbins=60)
fig.show(title="Cena nemovitosti v Iowa - Ames")

In [ ]:
# puvodni hodnoty
df_housing.loc[:, ['SalePrice']].head(10)

,SalePrice
0,208500
1,181500
2,223500
3,140000
4,250000
5,143000
6,307000
7,200000
8,129900
9,118000


In [ ]:
# logaritmizovane hodnoty
df_housing.loc[:, 'SalePrice'].transform(np.log)

,SalePrice
0,12.247694
1,12.109011
2,12.317167
3,11.849398
4,12.429216
...,...
1455,12.072541
1456,12.254863
1457,12.493130
1458,11.864462


## Chybějící a nejčastější hodnoty
Dalším z důležitých témat je nakládání s chybějícími hodnotami. Jejich počet si
snadno zobrazíme následovně

In [ ]:
df_housing.isna().mean().sort_values(ascending=False).head(15)

,0
PoolQC,0.995205
MiscFeature,0.963014
Alley,0.937671
Fence,0.807534
MasVnrType,0.597260
FireplaceQu,0.472603
LotFrontage,0.177397
GarageYrBlt,0.055479
GarageCond,0.055479
GarageType,0.055479


Některé sloupce označují velmi vysoké množství chybějících hodnot. Nastavíme tedy lit podílu chyvějcíích hodnot a takové sloupce odstraníme.

In [ ]:
threshold = 0.7

In [ ]:
df_new = df_housing[df_housing.columns[df_housing.isnull().mean() < threshold]]

In [ ]:
df_new.isna().mean().sort_values(ascending=False).head(15)

,0
MasVnrType,0.597260
FireplaceQu,0.472603
LotFrontage,0.177397
GarageCond,0.055479
GarageYrBlt,0.055479
GarageFinish,0.055479
GarageQual,0.055479
GarageType,0.055479
BsmtFinType2,0.026027
BsmtExposure,0.026027


Některé operace jous závislé na datovém typy. V následijící části si ukážeme, jak datové typy vyfiltrovat

In [ ]:
numeric_vars = df_housing.apply(lambda x: pd.api.types.is_numeric_dtype(x))

In [ ]:
df_categorical = df_housing.loc[:, ~numeric_vars]

Nejčetnější odnotu si zobrazíme následujícím způsobem

In [ ]:
df_categorical['MSZoning'].value_counts().idxmax()

'RL'

In [ ]:
df_categorical.iloc[:, :5].value_counts()

MSZoning  Street  Alley  LotShape  LandContour
RM        Pave    Grvl   Reg       Lvl            28
FV        Pave    Pave   Reg       Lvl            18
RL        Pave    Grvl   Reg       Lvl            11
RM        Pave    Pave   Reg       Lvl             5
FV        Pave    Pave   IR1       Lvl             5
RL        Pave    Pave   Reg       Lvl             4
RM        Pave    Grvl   Reg       Bnk             3
                         IR1       Lvl             2
RL        Pave    Grvl   Reg       Bnk             2
RM        Pave    Pave   Reg       Bnk             1
                         IR1       Lvl             1
                  Grvl   Reg       HLS             1
                         IR3       Bnk             1
C (all)   Pave    Pave   Reg       HLS             1
RM        Pave    Grvl   IR1       Bnk             1
C (all)   Pave    Pave   Reg       Lvl             1
RL        Pave    Pave   IR1       Bnk             1
                  Grvl   IR1       HLS             1
RH        Pave    Pave   Reg       Bnk             1
                         IR2       Bnk             1
FV        Pave    Pave   IR2       Lvl             1
RL        Pave    Pave   Reg       Bnk             1
Name: count, dtype: int64

Na numerických daových typech můžeme zobrazit popisné statistiky:

In [ ]:
df_housing.loc[:, numeric_vars].median()

,0
Id,730.5
MSSubClass,50.0
LotFrontage,69.0
LotArea,9478.5
OverallQual,6.0
OverallCond,5.0
YearBuilt,1973.0
YearRemodAdd,1994.0
MasVnrArea,0.0
BsmtFinSF1,383.5


## Odlehlé hodnoty a agregační funkce

Dalším ze způsobů filtrování datového souboru je detekce odlehlých hodnot.
Odlehlé hodnoty můžeme identifikovat mj. skrze kvantily standardizovaného normálního rozdělení:

In [ ]:
upper_lim = df_housing['SalePrice'].mean() + df_housing['SalePrice'].std() * 3

In [ ]:
upper_lim

419248.70453907084

In [ ]:
df_high_price = df_housing.loc[df_housing['SalePrice'] > upper_lim, :]

In [ ]:
df_high_price.loc[df_high_price['SalePrice'] < upper_lim, :]

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice


Další z často užívaných transformzcí je tzv. Dummy, nebo One-hot encoding, kterým jsme schopni převést kategorické proměnné na numerické.

In [ ]:
df_housing['MSZoning'].sample(15, random_state=42)

,MSZoning
892,RL
1105,RL
413,RM
522,RM
1036,RL
614,RM
218,RL
1160,RL
649,RM
887,RL


In [ ]:
pd.get_dummies(df_housing['MSZoning'].sample(15, random_state=42))

,C (all),RL,RM
892,False,True,False
1105,False,True,False
413,False,False,True
522,False,False,True
1036,False,True,False
614,False,False,True
218,False,True,False
1160,False,True,False
649,False,False,True
887,False,True,False


Č

In [ ]:
df_housing['SalePrice'].mean()

180921.19589041095

In [ ]:
df_housing.loc[:, ['MSZoning', 'SalePrice']].groupby(['MSZoning']).mean()

,SalePrice
MSZoning,
C (all),74528.000000
FV,214014.061538
RH,131558.375000
RL,191004.994787
RM,126316.830275


## Škálování dat

Pro některé algoritmy strojového učení je vhodné mít data převedená na stejou škálu. Jak škálování provádět se podíváme v této části

In [ ]:
df_scaled = df_housing.loc[:, ['YearBuilt', 'SalePrice']]

In [ ]:
px.histogram(df_scaled.melt(), facet_row='variable')

Vidíme, že data jsou na úplně rozdílné škále. Jak je tedy převíést na jednotnou škálu? Můžeme použít 2 základní druhý škálování = MinMax a Standard Scaling

In [ ]:
# pouzijeme knihovnu sklearn pro strojove uceni
from sklearn.preprocessing import MinMaxScaler, StandardScaler

In [ ]:
minmax_scaler = MinMaxScaler()
df_scaled_minmax = pd.DataFrame(minmax_scaler.fit_transform(df_scaled), columns=df_scaled.columns)

In [ ]:
std_scaler = StandardScaler()
df_scaled_std = pd.DataFrame(std_scaler.fit_transform(df_scaled), columns=df_scaled.columns)

In [ ]:
px.histogram(df_scaled_minmax.melt(), facet_row='variable')

In [ ]:
px.histogram(df_scaled_std.melt(), facet_row='variable')

Vidíme, že nyní jsou obě proměnné převedeny na stejnou škálu. Rozdíl mezi MinMax škáluje mezi minimální a maximální hodnoutu - hodnoty jsou tedy omezené. Oproti tomu StandardScaler škáluje podle hodnot standardizovaného normálního rozložení, které není omezeno.

## Práce s datumem

V této části si připomeneme, jak pracovat s daumem pomocí datetime modulu

In [ ]:
jmeno = "Stepan Vondracek"

In [ ]:
# pripomente si jak rozdelit string
jmeno.split(" ")

['Stepan', 'Vondracek']

In [ ]:
# vytvorte tabulku se jmeny
df_names = pd.DataFrame({"jmeno": ["Stepan Vondracek", "Karel Novotny"]})
df_names

,jmeno
0,Stepan Vondracek
1,Karel Novotny


In [ ]:
# chceme vybrat pouze vsechna krestni jmena - ta jsou v ramci stringu oddelena
# pomoci mezery. VYbirame tedy ty casti stringu pred mezerou
df_names['krestni'] = df_names['jmeno'].str.split(" ").apply(lambda x: x[0])

In [ ]:
df_names

,jmeno,krestni
0,Stepan Vondracek,Stepan
1,Karel Novotny,Karel


In [ ]:
# Vytvorime tabulku s datumem

from datetime import date
data = pd.DataFrame({'date':
['01-01-2017',
'04-12-2008',
'23-06-1988',
'25-08-1999',
'20-02-1993',
]})


In [ ]:
# podobne jako v predchozi casti muzeme extrahovat rok, ale je to nejlepsi zpusob?
data['year'] = data['date'].str.split("-").apply(lambda x: x[-1])

In [ ]:
# datovy typ je object coz odpovida typu string
data.dtypes

,0
date,object
year,object


In [ ]:
# prevedeme datum na datetime format
data['date'] = pd.to_datetime(data['date'], format="%d-%m-%Y")
data

,date,year
0,2017-01-01,2017
1,2008-12-04,2008
2,1988-06-23,1988
3,1999-08-25,1999
4,1993-02-20,1993


In [ ]:
# jiny zpusob extrakce roku za pouziti datetime typu
data['date'].dt.year

,date
0,2017
1,2008
2,1988
3,1999
4,1993


In [ ]:
from datetime import date
date_now = date.today()
date_now

datetime.date(2025, 2, 18)

In [ ]:
# s daty muzeme delat ruzne numericke operace, jako napriklad takhle spocitat dobu ktera uplynula dodnes od daneho data
date_now = pd.to_datetime('now')
data['years_passed_from_now'] = (date_now - data['date']).dt.days // 365

In [ ]:
data

,date,year,years_passed_from_now
0,2017-01-01,2017,8
1,2008-12-04,2008,16
2,1988-06-23,1988,36
3,1999-08-25,1999,25
4,1993-02-20,1993,32
